In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

np.set_printoptions(precision=3, linewidth=np.inf)
np.random.seed(2023)

# 线性回归的基本形式

大部分机器学习算法可以分为两大类: 有监督学习与无监督学习. 对于一批样本数据 $\{X_i\}_{i=1}^{n}$, 其中每一个 $X_i$ 是一个 $m$ 维向量, 用一个数据矩阵 $X\in\mathbb{R}^{n\times m}$ 来表示样本数据. **有监督学习**指的是每一个样本数据 $X_i$ 对应一个标签 $y_i$, 通常在实际生活中, 希望用观察到的 $X_i$ 预测未观察到的 $y_i$, 因此算法希望从一系列有标签数据中学到一个能够尽可能预测标签的范式 $h(X_i)=y_i$; **无监督学习**指的是样本数据没有标签, 希望根据样本数据的某些特征来达成一定的目标. 

在有监督学习问题中, 根据标签 $y_i$ 类型的不同, 又可以分为分类和回归两类: 分类指 $y_i$ 只取离散值的情况, 对应着样本所属的类别; 回归指 $y_i$ 可以在一个连续的范围内取值. 在线性回归中, 我们就需要根据看到的样本特征 $X_i$ 预测其标签 $\hat{y}_i$, 并希望预测值 $\hat{y}_i$ 尽可能地接近真实值 $y_i$.

任何学习算法都依赖于一定的假设, 而线性回归所依赖的假设形式十分简单: 它假设标签是特征的线性函数, 即

$$
y_i = w_0 + w_1X_{i1} + w_2X_{i2} + \cdots + w_mX_{im}=w_0+Xw,\quad w=(w_1,\cdots,w_n)^T\in\mathbb{R}^{n}.
$$

我们称一个能根据输入给出一定结果的函数为估计器. 线性回归是一个十分简单的估计器, 它的所有信息由参数 $w_0$ 和 $w$ 所定义, 确定了这两个值, 线性回归模型就是确定的. 至于如何根据给定的训练数据确定模型参数, 线性回归使用简单最小二乘法(OLS), 即希望预测结果与真实结果之间的残差平方和最小:

$$
(w_0,w)=\arg\min_{w_0,w}\sum_{i=1}^{n}\left(y_i-(w_0+X_iw) \right)^2.
$$

在sklearn中, 由`linear_model,LinearRegression`类可以创建一个线性回归实例, 用它来完成我们所需要的一切工作. 接下来我们结合数据集讨论, 使用sklearn中内置的`diabetes`数据集, 首先加载数据进行预览. `diabetes`是一个简单的回归数据集, 它包含了一共442个样本, 一共10个特征.

In [2]:
# help(load_diabetes)

X, y = load_diabetes(return_X_y=True)
print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

X.shape:  (442, 10)
y.shape:  (442,)


由于模型的训练会学到给定数据中内置的知识, 为了测试模型推广到未知的数据中是否还有足够好的标准, 我们一般会留下数据中的一部分不参与训练, 而是用来检测模型的效果. 将用于模型训练的数据称为**训练集**, 将用于检测模型效果的数据称为**测试集**, 它们的比例没有明确要求, 一般随机抽取以保证两个数据集中的特征与标签分布大致相同. 几乎任何模型训练都需要测试集来评估模型的效果, 不能让模型只对见过的数据有好的表现.

当然, 在拆分训练集和测试集的过程中, 要保证每一个样本的特征和标签是匹配的.

In [3]:
X_train, X_test = X[:300], X[300:]
y_train, y_test = y[:300], y[300:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(300, 10) (300,)
(142, 10) (142,)


现在, 获得了训练集和测试集后, 我们应当围绕下面的步骤来搭建一个回归模型. 注意, 下面的步骤几乎适用于任何有监督学习模型.

- 创建一个模型实例. 模型实例中包含了模型的基本信息, 这些信息在还未见到数据时就要确定.
- 根据数据拟合模型. 这一步从数据中学习知识, 得到一个可用模型, 学习的过程由模型实例所决定.
- 用拟合的模型预测训练集, 观察模型在训练集上的结果.
- 用拟合的模型预测测试集, 观察模型在测试集上的结果并对比.

第一步, 用`LinearRegression()`创建一个模型实例. 有很多模型会在创建实例时就指定好一系列需指定的参数, 而简单线性回归模型需要预先指定的参数不多, 一般情况下不需要传入任何参数, 只有在对模型有具体要求, 比如不需要常数截距, 或者需要系数全部非负时, 才会调整简单线性回归的参数.

In [4]:
model = LinearRegression()

接下来, 用数据拟合模型. 在线性回归模型中, 创建的实例有`.fit()`方法, 它接受数据(特征和标签)拟合模型, 这里只传入训练数据. 需注意, 模型拟合不是inplace的, 它不会直接改变创建的实例, 因此要用一个变量来接受拟合后的模型.

In [5]:
model = model.fit(X_train, y_train)

拟合过程是自动的, 这意味着我们不需要考虑模型会如何运用最小二乘法找到最优的参数. 拟合完毕后, 可以通过模型属性查看模型的具体信息. 在sklearn中, **属性**指的是一系列**以下划线结束的类属性**, 它允许我们了解模型的内部构造. 对简单线性回归而言, 可以查看以下属性[(1)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html): 

- `coef_`: 系数, 即上述公式中的 $w$, 是一个`(n_features,)`的向量.
- `intercept_`: 截距, 即上述公式中的 $w_0$, 是一个数.
- `rank_`: 矩阵`X`的秩.
- `singular_`: 矩阵`X`的奇异值.
- `n_features_in_`: 训练过程中看到的特征数.
- `feature_names_in_`: 训练过程中看到的特征名, 是一个`(n_features_in)`的字符串列表.

在线性回归模型中, 我们只关心模型的形式, 即我们如何根据特征得到预测值, 因此只需查看`coef_`和`intercept_`即可.

In [6]:
print('coef of linear regression: ', model.coef_)
print('intercept of linear regression: ', model.intercept_)

coef of linear regression:  [ -16.576 -254.665  560.986  278.918 -393.414   97.055  -19.002  169.465  632.951  114.216]
intercept of linear regression:  152.34786451820108


虽然我们能够获得模型参数, 从而得到模型的具体形式, 但自己写一个矩阵乘法来计算结果显然是很繁琐的. 所幸, 拟合后的模型提供了一系列方法, 帮助我们很方便地应用模型. 在`LinearRegression`中, 调用`fit`方法后, 模型的下述方法将可以使用:

- `predict`: 传入特征矩阵, 输出此特征矩阵对应的预测值.
- `score`: 传入特征矩阵及其对应的真实标签值, 返回模型在这组数据上的得分.

在简单线性回归中, 调用`predict`方法相当于自动执行了一遍 $\hat{y}_i=X_iw+w_0$, 从而省去了繁琐的乘法和加法步骤; 而调用`score`方法则是计算模型预测这组数据的 $R^2$: $R^2$ 是一个用于评估回归模型解释能力的指标, 其计算方式为

$$
R^2=1-\frac{\sum(y_i-\hat{y}_i)^2}{\sum(y_i-\bar y)^2}.
$$

$R^2$值越大, 代表模型的解释能力越强. 如取到最大值 $1$, 则说明预测值完全和真实值吻合导致被减数为 $0$; $R^2$ 也可以是负数, 这代表模型的预测能力甚至不如简单地用标签平均值预测.

下面我们用拟合得到的模型预测第一个数据点, 以及评估在训练集和测试集上的解释能力.

In [7]:
print("The prediction and true value of the first sample.")
print('pred: {}, true: {}'.format(model.predict([X[0]]), y[0]))

print("R2 of training data: {}".format(model.score(X_train, y_train)))
print("R2 of testing data: {}".format(model.score(X_test, y_test)))

The prediction and true value of the first sample.
pred: [204.511], true: 151.0
R2 of training data: 0.5147187604205686
R2 of testing data: 0.5071991852187347


可以发现, 此回归模型的解释能力大约为 $0.5$, 并且在训练集上的解释能力略高于测试集上. 一般模型在训练集上的表现要优于测试集, 因为模型倾向于过度学习训练集上的信息, 以期在训练集上获得尽可能好的表现, 但这往往以损失在真实分布上的表现为代价. 由于线性回归模型形式十分简单, 它过度学习的程度很轻, 因此在训练集上的表现仅是略高于测试集上; 而对于复杂的数据集, 或者变量较多的情况下, 如果不加以控制, 训练集上的表现可能要比测试集上好的多, 这种现象被称为**过拟合**.

下面, 我们将所有执行过的步骤合在一起, 给出用sklearn搭建模型的一般框架.

In [8]:
X, y = load_diabetes(return_X_y=True)
X_train, y_train = X[:300], y[:300]
X_test, y_test = X[300:], y[300:]

model = LinearRegression().fit(X_train, y_train)
print('R2 of training set: {}'.format(model.score(X_train, y_train)))
print('R2 of testing set: {}'.format(model.score(X_test, y_test)))

R2 of training set: 0.5147187604205686
R2 of testing set: 0.5071991852187347


使用sklearn搭建模型整体步骤就是这么简单, 简单得让人有些无从下手修改. 实际上, sklearn不只提供了这种高度简明的模型搭建过程, 其中的许多步骤是允许个性化的. 在日后的学习中, 我们会更多地介绍sklearn的其他功能.